In [ ]:
######################################################################################
# Lets see what people have to say about zomato's IPO by Applying some nlp skills    #
######################################################################################

# making neccessary import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [ ]:
# lets load and inspect the data
dataFrame = pd.read_csv('/kaggle/input/tweets-about-zomatoipo/zomato-ipo.csv')
dataFrame.info()

Lets perform some data cleaning to have a better insight!!
The parameters the seems relevant for our analysis are date, username (because lets be sure no one is creating hype, by tweeting a particular sentiment message), tweet, language, like_count, retweet count, hashtags(lets take these as well)

In [ ]:
cleanFrame = dataFrame[['date', 'username', 'tweet', 'language', 'likes_count', 'retweets_count', 'hashtags']]
cleanFrame.info()

In [ ]:
cleanFrame

Let's first make sure that the data is not baised anyhow by checking the unique usernames.

In [ ]:
print('A little summary is as follow:')
print('* There are {} unique users who tweeted '.format(len (set (cleanFrame['username']))))
print('* This means {} % tweets are unique'.format( 100 *len (set (cleanFrame['username'])) / len(cleanFrame)))
print('* of these tweets {} % were in English'.format( 100 *len ((cleanFrame[cleanFrame['language'] == 'en'])) / len(cleanFrame)))

maxLiked = max(dataFrame['likes_count'])
print()
print( '* The most like tweet was like {} times'.format(maxLiked))
print('The most liked tweet is as follows ahem:')
print(dataFrame[dataFrame['likes_count'] == maxLiked]['tweet'].values[0])
print()

maxRetweeted = max(dataFrame['retweets_count'])
print('* The most retweeted tweet was tweeted {} times'.format(maxRetweeted))
print('which read as follows: ')
print(dataFrame[dataFrame['retweets_count'] == maxRetweeted]['tweet'].values[0])

Before we proceed to word clouds and data clean lets go through the hashtags firts.

In [ ]:
# de-emojify 
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
## Assumption to keep this analysis simple we will only be analyzing englis tweets
cleanFrame = cleanFrame[cleanFrame['language'] == 'en']

cleanFrame['hashtags'] = cleanFrame['hashtags'].apply(lambda a : a.replace('[', ''))
cleanFrame['hashtags'] = cleanFrame['hashtags'].apply(lambda a : a.replace(']', ''))
cleanFrame['hashtags'] = cleanFrame['hashtags'].apply(lambda a : a.replace('\'', ''))
# Functions for deEmojifying
cleanFrame['hashtags'] = cleanFrame['hashtags'].apply(lambda a : deEmojify(a))
cleanFrame['hashtags'] = cleanFrame['hashtags'].apply(lambda a : re.sub("[^a-zA-Z0-9]+", ' ', a))

# removes hyperlinks
cleanFrame['hashtags'] = cleanFrame['hashtags'].apply(lambda a : re.sub(r'^https?:\/\/.*[\r\n]*', '', a, flags=re.MULTILINE))

In [ ]:
hashTags = []
for mem in cleanFrame['hashtags']:
    mems = mem.split(',')
    for member in mems:
        hashTags.append(member)

hashTags = list(set(hashTags))

In [ ]:
# lets start visualizations by plotting word clouds
# World Cloud Plotter
from wordcloud import WordCloud, STOPWORDS 
stopwords = set(STOPWORDS) 

def WordCloudForSentiments (corpus, title):
    wordcloud = WordCloud(width = 800, height = 800,background_color ='grey',
                          stopwords = stopwords,  min_font_size = 10).generate(corpus)
    
    plt.figure(figsize = (12, 12), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.rcParams.update({'font.size': 25})
    plt.axis("off") 
    plt.title('Word Cloud:  ' + title)
    plt.tight_layout(pad = 0) 
  
    plt.show() 

In [ ]:
WordCloudForSentiments( ''.join(hashTags),'WordCloud:hashtags' )

In [ ]:
# lets clean the tweets using the same functions above:
cleanFrame['tweet'] = cleanFrame['tweet'].apply(lambda a : deEmojify(a))
cleanFrame['tweet'] = cleanFrame['tweet'].apply(lambda a : re.sub("[^a-zA-Z0-9]+", ' ', a))
cleanFrame['tweet'] = cleanFrame['tweet'].apply(lambda a : re.sub(r'^https?:\/\/.*[\r\n]*', '', a, flags=re.MULTILINE))

In [ ]:
# with the tweets data cleaned lets build a corpus
tweetCorpus = ''
for tweet in cleanFrame['tweet']:
    tweetCorpus +=  (tweet + ' ')    # adding a space to seperate tweets

In [ ]:
#Plotting tweet corpus
WordCloudForSentiments( ''.join(tweetCorpus),'WordCloud:tweets' )

In [ ]:
#  A Function for Sentiment Analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def SentimentAnlysis(sentence):
    sentAnalyzer = SentimentIntensityAnalyzer() 
    sentDict = sentAnalyzer.polarity_scores(sentence)
    
    if sentDict['compound'] >= 0.05:
        return "positive"
    elif sentDict['compound'] <= -0.05 :
        return "negative"
    else:
        return "neutral"

In [ ]:
# lets do sentiment analyis now
from tqdm import tqdm
sentiment = []

for tweet in  tqdm (cleanFrame['tweet']):
    sentiment.append(SentimentAnlysis(tweet))

In [ ]:
# lets append this to our dataFrame and continue our analysis
cleanFrame['sentiments'] = sentiment

# A pie chart for understanding postive, negatives and neutral tweets

numPostives = len(cleanFrame[cleanFrame['sentiments'] == 'positive'])
numNegatives = len(cleanFrame[cleanFrame['sentiments'] == 'negative'])
numNeutrals  = len(cleanFrame[cleanFrame['sentiments'] == 'neutral'])

plt.figure(figsize = (7, 7))
plt.pie([numPostives, numNegatives, numNeutrals], labels = ['positives', 'negatives', 'neutrals'], autopct='%1.2f%%')
plt.title('Twitter Sentiments on Zomato IPO')

It seems folks have a pretty postive outlook for Zomato IPO, although I believe more neutrals should in be either postive
or negatives, but this essentially catches the sentiment.

In [ ]:
# lets do one thing more and try to see who the sentiment in changing with time
cleanFrame['date']= pd.to_datetime(cleanFrame['date'])

In [ ]:
# Now each unique date collect Positive, negative, and neutral tweets
dates = set(cleanFrame['date'])
positivesOndate = []
negativesOndate = []
neutralsOndate  = []
da              = []

for date in dates:
    pos = len (cleanFrame[(cleanFrame['date'] == date) & (cleanFrame['sentiments'] == 'positive')])
    neg = len (cleanFrame[(cleanFrame['date'] == date) & (cleanFrame['sentiments'] == 'negative')])
    neu = len (cleanFrame[(cleanFrame['date'] == date) & (cleanFrame['sentiments'] == 'neutral')])
    
    positivesOndate.append(pos)
    negativesOndate.append(neg)
    neutralsOndate.append(neu)
    da.append( str(date))

In [ ]:
# let plot this info
plt.figure(figsize=(10, 7))
plt.plot( positivesOndate, label = 'Postive Sentiments' )
plt.plot( negativesOndate, label = 'Negative Sentiments')
plt.plot( neutralsOndate, label = 'Neutral Sentiments ')
plt.ylabel('Number of tweets on that day')
plt.xlabel('Days passed')
plt.title('Zomato IPO Sentiment timeSeries')
plt.legend()
plt.show()
